In [ ]:
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy3
import plotly.graph_objs as go
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

#векторизация
from sklearn.feature_extraction.text import TfidfVectorizer

# #выделение тематик
# from sklearn.decomposition import LatentDirichletAllocation
# from gensim.corpora.dictionary import Dictionary
# from numpy import array
# import pyLDAvis
# import pyLDAvis.gensim_models as gensimvis
# import gensim

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/ML ITHUB осень/ДИ 2024/IT-СИП-22-23-DATA-ML/data/vacancies.xlsx')

In [ ]:
df.shape

(2706, 3)

In [ ]:
%%time
text=df['description']
norm_text=[]
morph=pymorphy3.MorphAnalyzer()
for s in text:
    s1 = re.sub(r'[^\w\s]+|[\d]+', r'',s).strip()
    s1 = s1.lower()
    s1 = word_tokenize(s1)
    words=[]
    for i in s1:
        pv = morph.parse(i)
        words.append(pv[0].normal_form)
    sentence=' '.join(words)
    norm_text.append(sentence)

CPU times: user 38 s, sys: 103 ms, total: 38.1 s
Wall time: 38.5 s


In [ ]:
russian_stopwords = stopwords.words("russian")
vectorizer_tfidf = TfidfVectorizer(max_features=500, min_df=20, max_df=0.7, stop_words=russian_stopwords)
text_tfidf = vectorizer_tfidf.fit_transform(norm_text)
text_tfidf = pd.DataFrame(text_tfidf.toarray(),columns=vectorizer_tfidf.get_feature_names_out())
text_tfidf.head()

,ajax,api,aspnet,backend,bitrix,bootstrap,cms,crm,css,delphi,...,читать,чпу,чтение,чужой,шаблон,эксплуатация,электронный,это,являться,язык
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046909


In [ ]:
text_tfidf.tail()

,ajax,api,aspnet,backend,bitrix,bootstrap,cms,crm,css,delphi,...,читать,чпу,чтение,чужой,шаблон,эксплуатация,электронный,это,являться,язык
2701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.140209,0.0,0.0,0.0,0.0,0.0,0.0,0.071549
2702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.123900
2704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


## Понижение размерности и визуализация

In [ ]:
!pip install umap-learn

#понижение размерности
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
def show_fig(new_shape):

    fig = go.Figure(go.Scatter3d(x=new_shape[:, 0],
                        y=new_shape[:, 1],
                        z=new_shape[:, 2],
                        marker=dict(opacity=0.9,
                                    reversescale=True,
                                    colorscale='Blues',
                                    size=5),
                        line=dict (width=0.02),
                        mode='markers',
                                 text=df['title']))
    fig.update_layout(
        scene = dict(
            xaxis = dict(nticks=4),
                         yaxis = dict(nticks=4),
                         zaxis = dict(nticks=4),),
        width=700,
        margin=dict(r=20, l=10, b=10, t=10))
    fig.show()

In [ ]:
pca=PCA(n_components=3)
reduced_pca = pca.fit_transform(text_tfidf)
show_fig(reduced_pca)

In [ ]:
tsne=TSNE(n_components=3)
reduced_tsne = tsne.fit_transform(text_tfidf)
show_fig(reduced_tsne)

In [ ]:
import umap
umap_emb = umap.UMAP(n_components=3)
reduced_umap = umap_emb.fit_transform(text_tfidf)
show_fig(reduced_umap)

## Выделение тематик

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(text_tfidf)
for i,topic in enumerate(LDA.components_):
    print(f'5 слов для темы #{i}:')
    print([vectorizer_tfidf.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print('\n')

5 слов для темы #0:
['программирование', 'разработка', 'предприятие', 'знание', 'работа', 'типовой', 'опыт', 'доработка', 'платформа', 'конфигурация']


5 слов для темы #1:
['mysql', 'javascript', 'работа', 'опыт', 'разработка', 'css', 'знание', 'html', 'php', 'сайт']


5 слов для темы #2:
['станок', 'микроконтроллер', 'высокий', 'групповой', 'тренировка', 'опыт', 'проведение', 'образование', 'работа', 'программа']


5 слов для темы #3:
['обеспечение', 'язык', 'sql', 'система', 'данные', 'программный', 'опыт', 'работа', 'знание', 'разработка']


5 слов для темы #4:
['направление', 'программа', 'работа', 'программирование', 'развиваться', 'образование', 'обучение', 'технический', 'ребёнок', 'желание']




In [ ]:
!pip install --upgrade gensim

In [ ]:
text_clean= []
for row in norm_text:
        text_clean.append(row.split())

from gensim.models import Phrases
bigram = Phrases(text_clean)
trigram = Phrases(bigram[text_clean])

for idx in range(len(text_clean)):
    for token in bigram[text_clean[idx]]:
        if '_' in token:

            text_clean[idx].append(token)
    for token in trigram[text_clean[idx]]:
        if '_' in token:
          text_clean[idx].append(token)

In [ ]:
from gensim.corpora.dictionary import Dictionary
from numpy import array
dictionary = Dictionary(text_clean)
dictionary.filter_extremes(no_below=10, no_above=0.1)
corpus = [dictionary.doc2bow(doc) for doc in text_clean]
print('Количество уникальных токенов: %d' % len(dictionary))
print('Количество документов: %d' % len(corpus))

Количество уникальных токенов: 1589
Количество документов: 2706


In [ ]:
from gensim.models.ldamulticore import LdaMulticore
model=LdaMulticore(corpus=corpus,id2word=dictionary, num_topics=2)
model.show_topics()

[(0,
  '0.008*"программный_продукт" + 0.006*"высокий_технический" + 0.006*"на_платформа" + 0.006*"база_данные" + 0.006*"умение_работать" + 0.006*"информационный_система" + 0.006*"программный_обеспечение" + 0.005*"типовой_конфигурация" + 0.005*"предприятие" + 0.004*"управление"'),
 (1,
  '0.011*"программный_обеспечение" + 0.007*"язык_программирование" + 0.007*"на_платформа" + 0.007*"база_данные" + 0.007*"умение_работать" + 0.005*"информационный_система" + 0.005*"html_css" + 0.005*"быть" + 0.004*"управление" + 0.004*"график_работа"')]

In [ ]:
!pip install fastapi kaleido python-multipart uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pip uninstall pandas
!pip install pandas==2.0.0

Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/pandas-2.2.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/pandas/*
Proceed (Y/n)? y
  Successfully uninstalled pandas-2.2.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 66.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.0.0 which is incompatible.
mizani 0.11.4 requires pandas>=2.1.0, but you have pandas 2.0.0 which is incompatible.
plotnine 0.13.6 requires pandas<3.0.0,>=2.1.0, but you have pandas 2.0.0 which is incompatible.
xarray 2024.9.0 requires pandas>=2.1, but you have pandas 2.0.0 which is incompatible.


In [ ]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.9 MB/s eta 0:00:00
^C


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import gensim
pyLDAvis.enable_notebook()
data = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(data)

/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1429: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
